## Problem set 1, Intro to NLP 2018

#### This is due on September 25, 2018, submitted electronically. 100 points total.

##### How to do this problem set:

- What version of Python should I use? 3.6!

- Most of these questions require writing Python code and computing results, and the rest of them have textual answers. To generate the answers, you will have to fill out a supporting file, `hw1.py`.

- For all of the textual answers you have to fill out have placeholder text which says "Answer in one or two sentences here." For each question, you need to replace "Answer in one or two sentences here" with your answer.

- Write all the answers in this ipython notebook. Once you are finished (1) Generate a PDF via (File -> Download As -> PDF) and upload to Gradescope (2)Turn in `hw_1.py` and `hw_1.ipynb` on Moodle.
  
- **Important** check your PDF before you turn it in to gradescope to make sure it exported correctly. If ipython notebook gets confused about your syntax it will sometimes terminate the PDF creation routine early. If your whole PDF does not print, try running `$jupyter nbconvert --to pdf 2018hw1.ipynb` to identify and fix any syntax errors that might be causing problems

- When creating your final version of the PDF to hand in, please do a fresh restart and execute every cell in order. Then you'll be sure it's actually right. One handy way to do this is by clicking `Cell -> Run All` in the notebook menu.
 
- This assignment is designed so that you can run all cells in a few minutes of computation time. If it is taking longer than that, you probably have made a mistake in your code.

##### Academic honesty 

- We will audit the Moodle code from a set number of students, chosen at random. The audits will check that the code you wrote and turned on Moodle generates the answers you turn in on your PDF. If you turn in correct answers on your PDF without code that actually generates those answers, we will consider this a serious case of cheating. See the course page for honesty policies.

- We will also run automatic checks of code on Moodle for plagiarism. Copying code from others is also considered a serious case of cheating.

In [3]:
# Run this cell! It sets some things up for you.

# This code makes plots appear inline in this document rather than in a new window.
import matplotlib.pyplot as plt
from __future__ import division  # this line is important to avoid unexpected behavior from division

# This code imports your work from hw_1.py
from model import *

%matplotlib inline
plt.rcParams['figure.figsize'] = (5, 4) # set default size of plots

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

#### BEGIN MODEL TRAINING ####
lil+pump-boss~124000000.txt
aer-wont+laugh~1800000.txt
kanye+west-blood+on+the+leaves~88000000.txt
nicki+minaj-chun+li~126000000.txt
tyler+the+creator-boredom~66000000.txt
childish+gambino-freaks+and+geeks~25000000.txt
jaden+smith-syre~4815000.txt
migos-motorsport~269000000.txt
aer-whatever+we+want~1001000.txt
chance+the+rapper-smoke+break~107000000.txt
chance+the+rapper-juke+jam~174000000.txt
tyler+the+creator-who+dat+boy+feat+aap+rocky~87000000.txt
lil+pump-jettski+grizzley~41000000.txt
meek+mill-on+me~3500000.txt
kanye+west-gold+digger~378000000.txt
chance+the+rapper-all+night~168000000.txt
kendrick+lamar-swimming+pools~218000000.txt
lil+pump-flex+like+ouu~109000000.txt
kodak+black-calling+my+spirit~1900000.txt
kanye+west-famous~312000000.txt
migos-narcos~172000000.txt
migos-notice+me~88000000.txt
aer-come+go~1776000.txt
migos-culture~31000000.txt
skee+lo-i+wish~36000000.txt
packy-wobbin~4582000.txt
drake-nonstop~318000000.txt
lil+pump-esskeetit~138000

In [6]:
PATH_TO_DATA = 'lyrics'

RAP_POS_LABEL = 'popular_rap'
RAP_NEG_LABEL = 'unpopular_rap'
COUNTRY_POS_LABEL = 'popular_country'
COUNTRY_NEG_LABEL = 'unpopular_country'

RAP_DIR = os.path.join(PATH_TO_DATA, "rap")
COUNTRY_DIR = os.path.join(PATH_TO_DATA, "country")

RAP_TRAIN_DIR = os.path.join(RAP_DIR, "train")
RAP_TEST_DIR = os.path.join(RAP_DIR, "test")
COUNTRY_TRAIN_DIR = os.path.join(COUNTRY_DIR, "train")
COUNTRY_TEST_DIR = os.path.join(COUNTRY_DIR, "test")

for label in [RAP_POS_LABEL, RAP_NEG_LABEL]:
    print(label)
    if label == RAP_POS_LABEL:
        print(os.listdir(RAP_TRAIN_DIR + "/" + label))
        for file in os.listdir(RAP_TRAIN_DIR + "/" + label):
            string = file.split('+')
            print(string)
        if len(os.listdir(RAP_TRAIN_DIR + "/" + label)) == 12:
            print("Great! You have 12 {} reviews in {}".format(label, RAP_TRAIN_DIR + "/" + label))
        else:
            print("Oh no! Something is wrong. Check your code which loads the reviews")
    else:
        print(os.listdir(RAP_TRAIN_DIR + "/" + label))
        if len(os.listdir(RAP_TRAIN_DIR + "/" + label)) == 7:
            print("Great! You have 7 {} reviews in {}".format(label, RAP_TRAIN_DIR + "/" + label))
        else:
            print("Oh no! Something is wrong. Check your code which loads the reviews")

popular_rap
['lil+pump-boss~124000000.txt', 'aer-wont+laugh~1800000.txt', 'kanye+west-blood+on+the+leaves~88000000.txt', 'nicki+minaj-chun+li~126000000.txt', 'tyler+the+creator-boredom~66000000.txt', 'childish+gambino-freaks+and+geeks~25000000.txt', 'jaden+smith-syre~4815000.txt', 'migos-motorsport~269000000.txt', 'aer-whatever+we+want~1001000.txt', 'chance+the+rapper-smoke+break~107000000.txt', 'chance+the+rapper-juke+jam~174000000.txt', 'tyler+the+creator-who+dat+boy+feat+aap+rocky~87000000.txt', 'lil+pump-jettski+grizzley~41000000.txt', 'meek+mill-on+me~3500000.txt', 'kanye+west-gold+digger~378000000.txt', 'chance+the+rapper-all+night~168000000.txt', 'kendrick+lamar-swimming+pools~218000000.txt', 'lil+pump-flex+like+ouu~109000000.txt', 'kodak+black-calling+my+spirit~1900000.txt', 'kanye+west-famous~312000000.txt', 'migos-narcos~172000000.txt', 'migos-notice+me~88000000.txt', 'aer-come+go~1776000.txt', 'migos-culture~31000000.txt', 'skee+lo-i+wish~36000000.txt', 'packy-wobbin~4582000

In [ ]:
# Actually reading the data you are working with is an important part of NLP! Let's look at one of these reviews

print (open(TRAIN_DIR + "/neg/98_1.txt").read())

## Part One: Intro to NLP in Python: types, tokens and Zipf's law

#### Types and tokens

One major part of any NLP project is word tokenization. Word tokenization is the task of segmenting text into individual words, called tokens. In this assignment, we will use simple whitespace tokenization. Take a look at the `tokenize_doc` function in `hw_1.py`. **You should not modify tokenize_doc** but make sure you understand what it is doing.

In [ ]:
# We have provided a tokenize_doc function in hw_1.py. Here is a short demo of how it works

d1 = "This SAMPLE doc has   words tHat  repeat repeat"
bow = tokenize_doc(d1)

assert bow['this'] == 1
assert bow['sample'] == 1
assert bow['doc'] == 1
assert bow['has'] == 1
assert bow['words'] == 1
assert bow['that'] == 1
assert bow['repeat'] == 2

bow2 = tokenize_doc("CMPSCI 585 is already my favorite class this semester!")
for b in bow2:
    print (b)

Now we are going to look at the word types and word tokens in the corpus.
Use the `word_counts` dictionary variable to store the count of each word in the corpus.
Use the `tokenize_doc` function to break documents into tokens. **You should not modify tokenize_doc** but make sure you understand what it is doing.

**Question 1.1 (5 points)**

Complete the cell below to fill out the `word_counts` dictionary variable. `word_counts` keeps track of how many times a word type appears across the corpus. For instance, `word_counts["movie"]` should store the number 61492 -- the count of how many times the word `movie` appears in the corpus.

In [ ]:
import glob
import codecs
from collections import Counter

word_counts = Counter() # Counters are often useful for NLP in python

corpus = ''
for label in [RAP_POS_LABEL, RAP_NEG_LABEL]:
    for directory in [RAP_TRAIN_DIR, RAP_TEST_DIR]:
        for fn in glob.glob(directory + "/" + label + "/*txt"):
            print(fn)
            doc = codecs.open(fn, encoding = "ISO-8859-1") # Open the file with UTF-8 encoding
            corpus += doc.read() + ' '
    
word_counts = Counter(tokenize_doc(corpus))

In [ ]:
# you should see 61492 instances of the word type "movie" in the corpus. 
if word_counts["movie"] == 61492:
    print ("yay! there are {} total instances of the word type movie in the corpus".format(word_counts["movie"]))
else:
    print ("hmm. Something seems off. Double check your code")

**Question 1.2 (5 points)**

Take a look at the following values:

In [ ]:
print ("there are {} word types in the corpus".format(n_word_types(word_counts)))
print ("there are {} word tokens in the corpus".format(n_word_tokens(word_counts)))

You should see a much higher number of tokens than types. Why is that?

Word types are the distinct words that appear in the corpus and the word tokens are the number of times each word type appears in the corpus. 

**Question 1.3 (5 points)**

Using the word_counts dictionary you just created, make a new dictionary called sorted_dict where the words are sorted according to their counts, in decending order:

In [ ]:
import operator
sorted_dict = sorted(word_counts.items(), key=operator.itemgetter(1), reverse=True)

Now print the first 30 values from sorted_dict. 

In [ ]:
for i in range(30):
    print(sorted_dict[i])


#### Zipf's Law

**Question 1.4 (5 points)**

In this section, you will verify a key statistical properties of text: [Zipf's Law](https://en.wikipedia.org/wiki/Zipf%27s_law).

Zipf's Law describes the relations between the frequency rank of words and frequency value of words.  For a word $w$, its frequency is inversely proportional to its rank:

$$count_w = K \frac{1}{rank_w}$$
or in other words
$$\log(count_w) = K - \log(rank_w)$$

for some constant $K$, specific to the corpus and how words are being defined.

Therefore, if Zipf's Law holds, after sorting the words descending on frequency, word frequency decreases in an approximately linear fashion under a log-log scale.

Please make such a log-log plot by ploting the rank versus frequency **Hint: Make use of the sorted dictionary you just created.**.  Use a scatter plot where the x-axis is the *log(rank)*, and y-axis is *log(frequency)*.  You should get this information from `word_counts`; for example, you can take the individual word counts and sort them.  dict methods `.items()` and/or `values()` may be useful.  (Note that it doesn't really matter whether ranks start at 1 or 0 in terms of how the plot comes out.) You can check your results by comparing your plots to ones on Wikipedia; they should look qualitatively similar.

*Please remember to label the meaning of the x-axis and y-axis.*

In [ ]:
import math
import operator
x = []
y = []
X_LABEL = "log(rank)"
Y_LABEL = "log(frequency)"

for i in range(len(sorted_dict)):
    x += [math.log(i+1)]
    y += [math.log(sorted_dict[i][1])]

plt.scatter(x, y)
plt.xlabel(X_LABEL)
plt.ylabel(Y_LABEL)

**Question 1.5 (5 points)**

You should see some discontinuities on the left and right sides of this figure.  Why are we seeing them on the left?  Why are we seeing them on the right?  On the right, what are those "ledges"?

We are seeing discontinuities on the left and right due to the nature of log growth. As x goes to 1 in log(x), the the growth of log(x) to log(x+1) is large. The ledges on the right are the infrequent words. There are many words that show up only once or twice in the text, and the large amounts of them create a ledge in the graph.

## Part Two: Naive Bayes

This section of the homework will walk you through coding a Naive Bayes classifier that can distinguish between positive and negative reviews (at some level of accuracy).

**Question 2.1 (10 pts) ** 

To start, implement the `update_model` function in `hw1.py`. Make sure to read the function comments so you know what to update. Also review the NaiveBayes class variables in the `def __init__` method of the NaiveBayes class  to get a sense of which statistics are important to keep track of. Once you have implemented `update_model`, run the train model function using the code below. What is the size of the vocabulary used in the training documents? You’ll need to provide the path to the dataset you downloaded to run the code.

In [14]:
nb = NaiveBayesTextClassification(PATH_TO_DATA, tokenizer=tokenize_doc)
nb.train_model()


if len(nb.vocab) == 251637:
    print ("Great! The vocabulary size is {}".format(251637))
else:
    print ("Oh no! Something seems off. Double check your code before continuing. Maybe a mistake in update_model?")

#### BEGIN MODEL TRAINING ####
REPORTING CORPUS STATISTICS
NUMBER OF RAP SONGS IN EXTREMELY POPULAR CLASS: 22.0
NUMBER OF RAP SONGS IN VERY POPULAR CLASS: 17.0
NUMBER OF RAP SONGS IN STANDARD POPULAR CLASS: 12.0
NUMBER OF RAP SONGS IN EXTREMELY UNPOPULAR CLASS: 26.0
NUMBER OF RAP SONGS IN VERY UNPOPULAR CLASS: 36.0
NUMBER OF RAP SONGS IN STANDARD UNPOPULAR CLASS: 37.0
NUMBER OF COUNTRY SONGS IN EXTREMELY POPULAR CLASS: 0.0
NUMBER OF COUNTRY SONGS IN VERY POPULAR CLASS: 0.0
NUMBER OF COUNTRY SONGS IN STANDARD POPULAR CLASS: 0.0
NUMBER OF COUNTRY SONGS IN EXTREMELY UNPOPULAR CLASS: 0.0
NUMBER OF COUNTRY SONGS IN VERY UNPOPULAR CLASS: 0.0
NUMBER OF COUNTRY SONGS IN STANDARD UNPOPULAR CLASS: 0.0
NUMBER OF TOKENS IN RAP EXTREMELY POPULAR CLASS: 12075.0
NUMBER OF TOKENS IN RAP VERY POPULAR CLASS: 9437.0
NUMBER OF TOKENS IN RAP STANDARD POPULAR CLASS: 5335.0
NUMBER OF TOKENS IN RAP EXTREMELY UNPOPULAR CLASS: 10908.0
NUMBER OF TOKENS IN RAP VERY UNPOPULAR CLASS: 15504.0
NUMBER OF TOKENS IN RAP

AttributeError: 'NaiveBayesTextClassification' object has no attribute 'vocab'

#### Exploratory analysis

Let’s begin to explore the count statistics stored by the update model function. Implement the provided `top_n` function to find the top 10 most common words in the positive class and top 10 most common words in the negative class. 

In [ ]:
print ("TOP 10 WORDS FOR CLASS " + RAP_POS_LABEL + ":")
for tok, count in nb.top_n(RAP_POS_LABEL, 10):
    print ('', tok, count)
print ()

print ("TOP 10 WORDS FOR CLASS " + RAP_NEG_LABEL + ":")
for tok, count in nb.top_n(RAP_NEG_LABEL, 10):
    print ('', tok, count)
print ()

**Question 2.2 (5 points)**

What is the first thing that you notice when you look at the top 10 words for the 2 classes? Are these words helpful for discriminating between the two classes? Do you think this trend carries forward to other texts from the English language? What about other languages?

The top 10 words for the 2 classes are practically identical. These words are not helpful for discriminating between the 2 classes. This trend carries to other texts in and out of the English language because there are many pronouns and prepositions that are commonly used throughtout all languages.

**Question 2.3 (10 pts) **

The Naive Bayes model assumes that all features are conditionally independent given the class label. For our purposes, this means that the probability of seeing a particular word in a document with class label $y$ is independent of the rest of the words in that document. Implement the `p_word_given_label` function. This function calculates P (w|y) (i.e., the probability of seeing word w in a document given the label of that document is y).

Use your `p_word_given_label` function to compute the probability of seeing the word “amazing” given each sentiment label. Repeat the computation for the word “dull.” 

In [ ]:
print ("P('amazing'|pos):",  nb.p_word_given_label("fuck", RAP_POS_LABEL))
print ("P('amazing'|neg):",  nb.p_word_given_label("fuck", RAP_NEG_LABEL))
print ("P('dull'|pos):",  nb.p_word_given_label("money", RAP_POS_LABEL))
print ("P('dull'|neg):",  nb.p_word_given_label("money", RAP_NEG_LABEL))

Which word has a higher probability, given the positive class? Which word has a higher probability, given the negative class? Is this behavior expected?

Given the positive class, 'amazing' has the higher probability. Given the negative class, 'dull' has the higher probability. This is expected since 'amazing' is usually associated with positive connotations while 'dull' is assiciated with negative connotations.

What is the purpose of the independence assumption for the Naive Bayes classifier?

In the Naive Bayes classifier, we are not worried about the order of words and their relations with other words. Because of this, we can assume that probabilites for these words are all independent of each other. 

**Question 2.4 (5 pts)**

In the next cell, compute the probability of the word "stop-sign." in the positive training data and negative training data.

In [ ]:
print ("P('stop-sign.'|pos):",  nb.p_word_given_label("fuck", RAP_POS_LABEL))
print ("P('stop-sign.'|neg):",  nb.p_word_given_label("fuck", RAP_NEG_LABEL))

What is unusual about P('stop-sign.'|pos)? Why is this a problem?

The word 'stop-sign.' does not come up in the pos class. However, 'stop-sign.' is ambiguous and does not hold that much of a relation to a positive or negative review. Thus, by our current probability, see the word 'stop-sign.' would automatically make the review a negative review even though it shows low significance. 

**Question 2.5 (5 pts)**

We can address the issues from question 2.4 with add-$\alpha$ smoothing (like add-1 smoothing except instead of adding 1 we add $\alpha$). Implement
`p_word_given_label_and_alpha` and then run the next cell. Hint: look at the slides from the lecture and the corresponding exercise on add-1 smoothing.

In [ ]:
print ("P('stop-sign.'|pos):",  nb.p_word_given_label_and_alpha("fuck", RAP_POS_LABEL, 0.2))

**Question 2.6 (5 pts)** 

*Prior and Likelihood* 

As noted before, the Naive Bayes model assumes that all words in a document are independent of one another given the document’s label. Because of this we can write the likelihood of a document as:

$P(w_{d1},\cdots,w_{dn}|y_d) = \prod_{i=1}^{n}P(w_{di}|y_d)$

However, if a document has a lot of words, the likelihood will become extremely small and we’ll encounter numerical underflow. Underflow is a common problem when dealing with probabilistic models; if you are unfamiliar with it, you can get a brief overview on [Wikipedia](https:/en.wikipedia.org/wiki/Arithmetic_underflow). To deal with underflow, a common transformation is to work in log-space.

$\log[P(w_{d1},\cdots,w_{dn}|y_d)] = \sum_{i=1}^{n}\log[P(w_{di}|y_d)]$

Implement the log_likelihood function (Hint: it should make calls to the p word given label and alpha function). Implement the log_prior function. This function takes a class label and returns the log of the fraction of the training documents that are of that label.

**Question 2.7 (5 pts) **

Naive Bayes is a model that tells us how to compute the posterior
probability of a document being of some label (i.e.,
$P(y_d|\mathbf{w_d})$).  Specifically, we do so using bayes rule:

  $P(y_d|\mathbf{w_d}) = \frac{P(y_d)P(\mathbf{w_d}|y_d)}{P(\mathbf{w_d})}$

In the previous section you implemented functions to compute both
the log prior ($\log[P(y_d)]$) and the log likelihood
($\log[P( \mathbf{w_d} |y_d)]$ ). Now, all you're missing is the
*normalizer*, $P(\mathbf{w_d})$.

Derive the normalizer by expanding $P(\mathbf{w_d})$.

$P(w_d) = \prod_{i=0}^{n} P(w_{i})$ 

**Question 2.8 (5 pts)**

One way to classify a document is to compute the unnormalized log posterior for both labels and take the argmax (i.e., the label that yields the higher unnormalized log posterior). The unnormalized log posterior is the sum of the log prior and the log likelihood of the document. Why don’t we need to compute the log normalizer here?

Since we are calculating probabilites for individual words and not phrases, there are no dependencies for these probabilities. Additionally, order does not matter during out calculations. Thus, all words are independent of each toher and there is no need to normalize the log posterior.

**Question 2.9 (5 pts)**

As we saw earlier, the top 10 words from each class do not give us much to go on when classifying a document. A much more powerful metric is the likelihood ratio, which is defined as

$LR(w)=\frac{P(w|y=\mathrm{pos})}{P(w|y=\mathrm{neg})}$

A word with LR 3 is 3 times more likely to appear in the positive class than in the negative. A word with LR 0.3 is one-third as likely to appear in the positive class as opposed to the negative class.

In [ ]:
# Implement the nb.likelihood_ratio function and use it to investigate the likelihood ratio of "amazing" and "dull"
print ("LIKELIHOOD RATIO OF 'amazing':", nb.likelihood_ratio('fuck', 0.2))
print ("LIKELIHOOD RATIO OF 'dull':", nb.likelihood_ratio('funk', 0.2))
print ("LIKELIHOOD RATIO OF 'and':", nb.likelihood_ratio('police', 0.2))
print ("LIKELIHOOD RATIO OF 'to':", nb.likelihood_ratio('her', 0.2))

What is the minimum and maximum possible values the likelihood ratio can take?

The range for the likelihood ratio is (0, $\infty$). The likelihood ratio will approach 0, but cannot be 0 because we are applying add-alpha smoothing to the probabilities.

Find the word in the vocabulary with the highest likelihood ratio below.

In [ ]:
highest = -1
word = None
for n in nb.vocab:
    temp = nb.likelihood_ratio(n, 0.2)
    if temp > highest: 
        highest = temp
        word = n
print (word, highest)

**Question 2.10 (5 pts)**

The unnormalized log posterior is the sum of the log prior and the log likelihood of the document. Implement the `unnormalized_log_posterior` function and the `classify` function. The `classify` function should use the unnormalized log posteriors but should not compute the normalizer. Once you implement the `classify` function, we'd like to evaluate its accuracy.

In [11]:
print (nb.evaluate_classifier_accuracy('rap', 0.2))

#### BEGIN MODEL TRAINING ####
lil+pump-boss~124000000.txt
aer-wont+laugh~1800000.txt
kanye+west-blood+on+the+leaves~88000000.txt
nicki+minaj-chun+li~126000000.txt
tyler+the+creator-boredom~66000000.txt
childish+gambino-freaks+and+geeks~25000000.txt
jaden+smith-syre~4815000.txt
migos-motorsport~269000000.txt
aer-whatever+we+want~1001000.txt
chance+the+rapper-smoke+break~107000000.txt
chance+the+rapper-juke+jam~174000000.txt
tyler+the+creator-who+dat+boy+feat+aap+rocky~87000000.txt
lil+pump-jettski+grizzley~41000000.txt
meek+mill-on+me~3500000.txt
kanye+west-gold+digger~378000000.txt
chance+the+rapper-all+night~168000000.txt
kendrick+lamar-swimming+pools~218000000.txt
lil+pump-flex+like+ouu~109000000.txt
kodak+black-calling+my+spirit~1900000.txt
kanye+west-famous~312000000.txt
migos-narcos~172000000.txt
migos-notice+me~88000000.txt
aer-come+go~1776000.txt
migos-culture~31000000.txt
skee+lo-i+wish~36000000.txt
packy-wobbin~4582000.txt
drake-nonstop~318000000.txt
lil+pump-esskeetit~138000

**Question 2.11 (5 pts)**

Try evaluating your model again with a smoothing parameter of 1000.

In [13]:
print (nb.evaluate_classifier_accuracy('rap', 1000.0))

43.39622641509434


Does the accuracy go up or down when the pseudo count parameter is raised to 1000? Why do you think this is?

The accuracy goes down when we raise the pseudo count paramater to 1000. As we raise the pseudo count parameter to 1000, we are slowly over-estimating for values that would normally have a low probability. By giving these values over-estimated probabilities, we are decreasing our overall accuracy.

** Question 2.12 (5 pts)** 

Find a review that your classifier got wrong. 

In [ ]:
alpha = 1.0
pos_path = os.path.join(nb.rap_test_dir, RAP_POS_LABEL)
neg_path = os.path.join(nb.rap_test_dir, RAP_NEG_LABEL)
for (p, label) in [ (pos_path, RAP_POS_LABEL), (neg_path, RAP_NEG_LABEL) ]:
    for f in os.listdir(p):
        with open(os.path.join(p,f),'r') as doc:
            content = doc.read()
            bow = nb.tokenize_doc(content)
            classify_label = nb.classify(bow, alpha)
            if classify_label != label:
                print ('Label for the review: {}'.format(label))
                print ('Classify_Label for the review: {}'.format(classify_label))
                print (content)
                break
            break

What are two reasons your system might have misclassified this example? What improvements could you make that may help your system classify this example correctly?

This review uses many words and phrases associated with positive connotations. Words like 'good', 'entertain', and 'potential' to name a few. Additionally, this review directs most of the positive statements to the actors, not the movie. Thus, many of these positive comments are geared towards the actors' abilites and not the writing of the movie. A way to improve our system to add context to words by looking at phrases or even whole sentences. This way the whole meaning of phrases and sentences can be understood instead of potentially misleading words. 

**Question 2.13 (5 pts)**

Often times we care about multi-class classification rather than binary classification.

How many counts would we need to keep track of if the model were modified to support 5-class classification?

There would be 16 counts. 1 count for vocab, 5 for class_total_doc_counts, 5 for class_total_word_counts, and 5 for class_word_counts.

What would be the new decision rule (i.e., how would the classify function change)?

Instead of a binary classify function, there would need to be 5 checks for which class produced the maximum unnormalized_log_posterior. 